In [26]:
# Core libraries
import numpy as np
import pandas as pd

# System 
from pathlib import Path
import subprocess
from shutil import copy
import datetime

# Simulation
import tellurium as te
import roadrunner

In [2]:
%load_ext watermark

In [3]:
%watermark --iversions

roadrunner1.4.24
pandas    0.22.0
tellurium 2.1.3
numpy     1.14.3



### Setup paths and helper functions

In [9]:
models_path = Path("../data/models")
simulation_path = Path("../data/simulation_results")

In [27]:
def run_matlab_script(script_path):
    # script_path - Path object to the m script
    str_script = str((script_path).resolve())
    run_command = f""" -r "run('{str_script}');exit;" """

    matlab_run = subprocess.run(
        ["matlab", "-nodisplay", "-nosplash", "-nodesktop", run_command],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )

    # check for errors
    if matlab_run.returncode == 0:
        print("Run ended succesfully")
    else:
        print("Run ended with an error")

    return matlab_run.stdout.decode("UTF-8")

# Simulations of Chassagnole model

## Define the model and experimental setup

In [23]:
model = te.loadAntimonyModel(
    te.sbmlToAntimony(str(models_path / "original_files" / "Chassagnole2002.xml"))
)

In [5]:
exp_list = [
    {"sample_id": "Delta_tpi", "modifications": ["vTIS_rmaxTIS"]},
    # {"sample_id": "Delta_aAkgdh", "modifications": ["LPD_Vmax"]},
    {"sample_id": "Delta_fba", "modifications": ["vALDO_rmaxALDO"]},
    {"sample_id": "Delta_zwf", "modifications": ["vG6PDH_rmaxG6PDH"]},
    #{"sample_id": "Delta_pts", "modifications": ["vPTS_rmaxPTS"]},
    {"sample_id": "Delta_gnd", "modifications": ["vPGDH_rmaxPGDH"]},
    {"sample_id": "Delta_pfk", "modifications": ["vPFK_rmaxPFK"]},
    {"sample_id": "Delta_pgi", "modifications": ["vPGI_rmaxPGI"]},
    # {"sample_id": "Delta_pgl", "modifications": ["PGL_Vmax"]},
    #{"sample_id": "Delta_pps", "modifications": ["PPS_Vmax"]},
    {"sample_id": "Delta_pyk", "modifications": ["vPK_rmaxPK"]},
    {"sample_id": "Delta_rpe", "modifications": ["vRu5P_rmaxRu5P"]},
    {"sample_id": "Delta_rpi", "modifications": ["vR5PI_rmaxR5PI"]},
    # {"sample_id": "Delta_sdh", "modifications": ["SDH_Vmax"]},
    {"sample_id": "Delta_tal", "modifications": ["vTA_rmaxTA"]},
    {"sample_id": "Delta_tkt1", "modifications": ["vTKA_rmaxTKa"]},
    {"sample_id": "Delta_tkt2", "modifications": ["vTKB_rmaxTKb"]},
    #{"sample_id": "Delta_fbp", "modifications": ["FBP_Vmax"]},
]

## Run the simulations

In [ ]:
model.simulate(0, 1000, 20)

ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()


ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Chassagnole" / f"Chassagnole_result_WT.csv")

In [ ]:
for experiment in exp_list:
    # set the original state
    model.resetToOrigin()

    sample_id = experiment["sample_id"]
    print(f"Working on sample {sample_id}")

    for mod in experiment["modifications"]:
        # perform poor-mans 'knockout'
        model.setValue(mod, 0)

    # Instead of steady-state solver use long integration.
    # make the "fair" comparison because all other models run not to steady state
    model.simulate(0, 1000, 20)    

    ss_flux_ids = model.getReactionIds()
    ss_flux = model.getReactionRates()


    ids = ss_flux_ids
    values = ss_flux
    df = pd.DataFrame({"ID": ids, "Value": values})
    df.to_csv(simulation_path / "Chassagnole" / f"Chassagnole_result_{sample_id}.csv")

# Simulations of Millard model

## Define the model and experimental setup

In [22]:
# load as Antimony bypasses the lack of ability to change local parameters in native SBML model
model = te.loadAntimonyModel(
    te.sbmlToAntimony(
        str(models_path / "original_files" / "Millard_2017_MODEL1505110000_url.xml")
    )
)

In [7]:
exp_list = [
    {"sample_id": "Delta_tpi", "modifications": ["TPI_Vmax"]},
    {"sample_id": "Delta_aAkgdh", "modifications": ["LPD_Vmax"]},
    {"sample_id": "Delta_fba", "modifications": ["FBA_Vmax"]},
    {"sample_id": "Delta_zwf", "modifications": ["ZWF_Vmax"]},
    {"sample_id": "Delta_pts", "modifications": ["PTS_4_kF", "PTS_4_kR"]},
    {"sample_id": "Delta_gnd", "modifications": ["GND_Vmax"]},
    {"sample_id": "Delta_pfk", "modifications": ["PFK_Vmax"]},
    {"sample_id": "Delta_pgi", "modifications": ["PGI_Vmax"]},
    {"sample_id": "Delta_pgl", "modifications": ["PGL_Vmax"]},
    {"sample_id": "Delta_pps", "modifications": ["PPS_Vmax"]},
    {"sample_id": "Delta_pyk", "modifications": ["PYK_Vmax"]},
    {"sample_id": "Delta_rpe", "modifications": ["RPE_Vmax"]},
    {"sample_id": "Delta_rpi", "modifications": ["RPI_Vmax"]},
    {"sample_id": "Delta_sdh", "modifications": ["SDH_Vmax"]},
    {
        "sample_id": "Delta_tal",
        "modifications": ["F6P_GAP_TAL_kcat", "S7P_E4P_TAL_kcat"],
    },
    {
        "sample_id": "Delta_tkt1",
        "modifications": ["X5P_GAP_TKT_kcat", "S7P_R5P_TKT_kcat"],
    },
    {"sample_id": "Delta_tkt2", "modifications": ["F6P_E4P_TKT_kcat"]},
    {"sample_id": "Delta_fbp", "modifications": ["FBP_Vmax"]},
]

## Run the simulations
Simulations are being performed as ODEs integration for 1e5 time units. Results after the end of simulation are being used as "steady-state" values.

In [6]:
# WT simulation
model.resetToOrigin()

# For Dilution rate approx. equal to 0.2 h-1
model.FEED = 0.4

model.simulate(0, 10000)

ss_flux_ids = model.getReactionIds()
ss_flux = model.getReactionRates()

ids = ss_flux_ids
values = ss_flux
df = pd.DataFrame({"ID": ids, "Value": values})
df.to_csv(simulation_path / "Millard" /  f"Millard_result_WT.csv")

In [4]:
for experiment in exp_list:
    # set the original state
    model.resetToOrigin()
    # For Dilution rate approx. equal to 0.2 h-1
    model.FEED = 0.4
    
    sample_id = experiment["sample_id"]
    print(f"Working on sample {sample_id}")

    for mod in experiment["modifications"]:
        # perform poor-mans 'knockout'
        model.setValue(mod, 0)

    # Instead of steady-state solver use long integration.
    # make the "fair" comparison because all other models run not to steady state
    
    model.simulate(0, 10000)
 
    ss_flux_ids = model.getReactionIds()
    ss_flux = model.getReactionRates()

    ids = ss_flux_ids
    values = ss_flux
    df = pd.DataFrame({"ID": ids, "Value": values})
    df.to_csv(simulation_path / "Millard" /  f"Millard_result_{sample_id}.csv")

Working on sample Delta_tpi
Working on sample Delta_aAkgdh
Working on sample Delta_fba
Working on sample Delta_zwf
Working on sample Delta_pts
Working on sample Delta_gnd
Working on sample Delta_pfk
Working on sample Delta_pgi
Working on sample Delta_pgl
Working on sample Delta_pps
Working on sample Delta_pyk
Working on sample Delta_rpe
Working on sample Delta_rpi
Working on sample Delta_sdh
Working on sample Delta_tal
Working on sample Delta_tkt1
Working on sample Delta_tkt2
Working on sample Delta_fbp


# Simulations of Khodayari model
It's MATLAB based model, so the simulation is being performed by invoking matlab in CLI mode.

In [43]:
khod_path = models_path / "modified_files" / "Khodayari"

In [8]:
ko_script = khod_path / "Run_Khodayari_Continuous.m"
result = run_matlab_script(script_path = ko_script)

NameError: name 'models_path' is not defined

Move simulation results to `simulation_results`

In [25]:
destination = simulation_path / "Khodayari" / f"knockout_simulation"
destination.mkdir(parents=True, exist_ok=True)
for file in khod_path.glob("result_cont*.mat"):
    copy(file, destination)

[PosixPath('../data/models/modified_files/Khodayari/result_cont_WT.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_pgl.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_sdhCD.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_rpe.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_ppsA.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_fbaAB.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_pgi.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_pfkAB.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_tpi.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_fbp.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_gnd.mat'),
 PosixPath('../data/models/modified_files/Khodayari/result_cont_Delta_rpiAB.mat'),
 PosixPath('../data/models

# Simulations of Kurata model
It's MATLAB based model, so the simulation is being performed by invoking matlab in CLI mode.

In [44]:
kurata_path = models_path / "modified_files" / "Kurata_continuous"

In [ ]:
ko_script = kurata_path / "Run_Kurata_Continuous.m"
result = run_matlab_script(script_path = ko_script)

Move simulation results to `simulation_results`

In [ ]:
destination = simulation_path / "Kurata" / f"knockout_simulation"
destination.mkdir(parents=True, exist_ok=True)
for file in khod_path.glob("result_cont*.mat"):
    copy(file, destination)